In [19]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

RESULTADO_DIR = Path(r"E:\proyecto_agric_precision\Dataset de Imagenes\ortomosaicos_canales\resultado")
DATES = ["20251002", "20251103", "20251210"]

csv_paths = {d: RESULTADO_DIR / f"features_{d}.csv" for d in DATES}
ATTR_XLSX = RESULTADO_DIR / "shape2025att.xlsx"

OUT_DIR = RESULTADO_DIR / "PASO_09_temporal_3fechas_obsUnitId"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("OUT_DIR:", OUT_DIR)
print("CSV exists:", {d: p.exists() for d, p in csv_paths.items()})
print("ATTR exists:", ATTR_XLSX.exists(), ATTR_XLSX)


OUT_DIR: E:\proyecto_agric_precision\Dataset de Imagenes\ortomosaicos_canales\resultado\PASO_09_temporal_3fechas_obsUnitId
CSV exists: {'20251002': True, '20251103': True, '20251210': True}
ATTR exists: True E:\proyecto_agric_precision\Dataset de Imagenes\ortomosaicos_canales\resultado\shape2025att.xlsx


In [20]:
def find_plot_id_col(df):
    for c in df.columns:
        if str(c).lower() in ["plot_id", "plotid", "plot", "id"]:
            return c
    for c in df.columns:
        if "plot" in str(c).lower() and "id" in str(c).lower():
            return c
    raise ValueError(f"No encuentro plot_id. Columnas: {list(df.columns)}")

dfs = []
for d, p in csv_paths.items():
    if not p.exists():
        raise FileNotFoundError(f"No existe: {p}")

    df = pd.read_csv(p)
    pid = find_plot_id_col(df)
    df = df.rename(columns={pid: "plot_id"}).copy()
    df["date"] = d

    # convierte a numérico (lo no-numérico -> NaN)
    for c in df.columns:
        if c not in ["plot_id", "date"]:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    dfs.append(df)

feat_long = pd.concat(dfs, ignore_index=True)
print("✅ feat_long:", feat_long.shape)
display(feat_long.head())


✅ feat_long: (606, 11)


,plot_id,VF,NDVI_mean,SR_mean,CHLGR_mean,MARI_mean,OSAVI_mean,SAVI2_mean,valid_pixels,total_pixels,date
0,1,0.234843,0.876604,15.362690,4.458802,5.038614,0.663180,0.004765,22423,95481,20251002
1,2,0.156293,0.899277,19.509604,5.206114,6.521187,0.680468,0.004385,14923,95481,20251002
2,3,0.211330,0.891328,17.635601,5.469518,5.774983,0.710884,0.005210,20178,95481,20251002
3,4,0.158639,0.883821,16.541691,5.090737,5.251598,0.699957,0.005356,15098,95172,20251002
4,5,0.216232,0.873134,15.007159,4.791890,4.923483,0.688334,0.005298,20646,95481,20251002


In [21]:
attr = pd.read_excel(ATTR_XLSX)

if "plot_id" not in attr.columns or "obsUnitId" not in attr.columns:
    raise ValueError(f"El Excel debe tener plot_id y obsUnitId. Columnas: {list(attr.columns)}")

attr_small = attr[["plot_id", "obsUnitId"]].copy()
attr_small["obsUnitId"] = attr_small["obsUnitId"].astype(str)

data = feat_long.merge(attr_small, on="plot_id", how="left")

# Variables (las tuyas reales)
vars_present = ["VF", "NDVI_mean", "SR_mean", "CHLGR_mean", "MARI_mean", "OSAVI_mean", "SAVI2_mean"]
vars_present = [v for v in vars_present if v in data.columns]

print("✅ data:", data.shape)
print("obsUnitId nulos:", data["obsUnitId"].isna().sum())
print("✅ Variables:", vars_present)

# Export LONG
out_long = OUT_DIR / "dataset_long_3dates.csv"
data.to_csv(out_long, index=False)
print("✅ Guardado:", out_long)


✅ data: (606, 12)
obsUnitId nulos: 0
✅ Variables: ['VF', 'NDVI_mean', 'SR_mean', 'CHLGR_mean', 'MARI_mean', 'OSAVI_mean', 'SAVI2_mean']
✅ Guardado: E:\proyecto_agric_precision\Dataset de Imagenes\ortomosaicos_canales\resultado\PASO_09_temporal_3fechas_obsUnitId\dataset_long_3dates.csv


In [22]:
order_ids = sorted(data["obsUnitId"].dropna().unique())

# Boxplots por obsUnitId y fecha
for v in vars_present:
    plt.figure(figsize=(16,4))
    sns.boxplot(data=data, x="obsUnitId", y=v, hue="date", order=order_ids)
    plt.title(f"{v} por obsUnitId y fecha")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(OUT_DIR / f"box_{v}_by_obsUnitId.png", dpi=200)
    plt.close()

# Curvas de medias por obsUnitId vs fecha
for v in vars_present:
    tmp = (data.groupby(["date","obsUnitId"])[v]
           .mean()
           .reset_index()
           .sort_values(["obsUnitId","date"]))

    plt.figure(figsize=(10,4))
    sns.lineplot(data=tmp, x="date", y=v, hue="obsUnitId", marker="o")
    plt.title(f"Media de {v} por obsUnitId vs fecha")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(OUT_DIR / f"mean_{v}_obsUnitId_over_time.png", dpi=200)
    plt.close()

print("✅ Gráficas guardadas en:", OUT_DIR)


C:\Users\Admin\AppData\Local\Temp\ipykernel_19436\1396885887.py:9: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.tight_layout()
C:\Users\Admin\AppData\Local\Temp\ipykernel_19436\1396885887.py:10: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig(OUT_DIR / f"box_{v}_by_obsUnitId.png", dpi=200)
C:\Users\Admin\AppData\Local\Temp\ipykernel_19436\1396885887.py:9: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.tight_layout()
C:\Users\Admin\AppData\Local\Temp\ipykernel_19436\1396885887.py:10: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig(OUT_DIR / f"box_{v}_by_obsUnitId.png", dpi=200)
C:\Users\Admin\AppData\Local\Temp\ipykernel_19436\1396885887.py:10: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig(OUT_DIR / f"box_{v}_by_obsUnitId.png", dpi=200)
C:\Users\

✅ Gráficas guardadas en: E:\proyecto_agric_precision\Dataset de Imagenes\ortomosaicos_canales\resultado\PASO_09_temporal_3fechas_obsUnitId


In [23]:
# WIDE: una fila por plot_id/obsUnitId, columnas por fecha
wide = data.pivot_table(index=["plot_id","obsUnitId"], columns="date", values=vars_present)
wide.columns = [f"{var}_{date}" for var, date in wide.columns]
wide = wide.reset_index()

# DELTAS: Nov-Oct y Dic-Nov
d0, d1, d2 = DATES
deltas = wide[["plot_id","obsUnitId"]].copy()

for v in vars_present:
    c0, c1, c2 = f"{v}_{d0}", f"{v}_{d1}", f"{v}_{d2}"

    if c0 in wide.columns and c1 in wide.columns:
        deltas[f"delta_{v}_{d1}_{d0}"] = wide[c1] - wide[c0]
        deltas[f"pct_{v}_{d1}_{d0}"] = (wide[c1] - wide[c0]) / wide[c0].replace(0, np.nan) * 100

    if c1 in wide.columns and c2 in wide.columns:
        deltas[f"delta_{v}_{d2}_{d1}"] = wide[c2] - wide[c1]
        deltas[f"pct_{v}_{d2}_{d1}"] = (wide[c2] - wide[c1]) / wide[c1].replace(0, np.nan) * 100

# Guardar
out_wide_csv = OUT_DIR / "dataset_wide_3dates.csv"
out_wide_xlsx = OUT_DIR / "dataset_wide_3dates.xlsx"
out_deltas = OUT_DIR / "dataset_deltas_3dates.csv"

wide.to_csv(out_wide_csv, index=False)
wide.to_excel(out_wide_xlsx, index=False)
deltas.to_csv(out_deltas, index=False)

print("✅ Guardado:", out_wide_csv)
print("✅ Guardado:", out_wide_xlsx)
print("✅ Guardado:", out_deltas)

display(wide.head())
display(deltas.head())


✅ Guardado: E:\proyecto_agric_precision\Dataset de Imagenes\ortomosaicos_canales\resultado\PASO_09_temporal_3fechas_obsUnitId\dataset_wide_3dates.csv
✅ Guardado: E:\proyecto_agric_precision\Dataset de Imagenes\ortomosaicos_canales\resultado\PASO_09_temporal_3fechas_obsUnitId\dataset_wide_3dates.xlsx
✅ Guardado: E:\proyecto_agric_precision\Dataset de Imagenes\ortomosaicos_canales\resultado\PASO_09_temporal_3fechas_obsUnitId\dataset_deltas_3dates.csv


,plot_id,obsUnitId,CHLGR_mean_20251002,CHLGR_mean_20251103,CHLGR_mean_20251210,MARI_mean_20251002,MARI_mean_20251103,MARI_mean_20251210,NDVI_mean_20251002,NDVI_mean_20251103,...,OSAVI_mean_20251210,SAVI2_mean_20251002,SAVI2_mean_20251103,SAVI2_mean_20251210,SR_mean_20251002,SR_mean_20251103,SR_mean_20251210,VF_20251002,VF_20251103,VF_20251210
0,1,1,4.458802,5.138012,2.853492,5.038614,7.177726,4.443617,0.876604,0.886830,...,0.544906,0.004765,0.003900,3.851346e-03,15.362690,17.003988,6.728218,0.234843,0.346904,0.091534
1,2,2,5.206114,4.734136,3.428803,6.521187,5.317302,3.839409,0.899277,0.852209,...,0.611442,0.004385,0.004334,4.959710e-03,19.509604,12.621037,9.198384,0.156293,0.345024,0.096375
2,3,3,5.469518,6.041914,3.069994,5.774983,7.422002,27505.001953,0.891328,0.844865,...,0.000487,0.005210,0.004433,6.130523e-07,17.635601,11.901986,8.733396,0.211330,0.317738,0.134983
3,4,4,5.090737,2.946303,2.614598,5.251598,2.873740,24947.978516,0.883821,0.688718,...,0.000439,0.005356,0.004439,5.749103e-07,16.541691,5.425055,6.849193,0.158639,0.111688,0.095709
4,5,5,4.791890,6.231022,3.508522,4.923483,7.333121,37030.667969,0.873134,0.807354,...,0.000425,0.005298,0.004632,5.173079e-07,15.007159,9.394213,9.851377,0.216232,0.382168,0.121769


,plot_id,obsUnitId,delta_VF_20251103_20251002,pct_VF_20251103_20251002,delta_VF_20251210_20251103,pct_VF_20251210_20251103,delta_NDVI_mean_20251103_20251002,pct_NDVI_mean_20251103_20251002,delta_NDVI_mean_20251210_20251103,pct_NDVI_mean_20251210_20251103,...,delta_MARI_mean_20251210_20251103,pct_MARI_mean_20251210_20251103,delta_OSAVI_mean_20251103_20251002,pct_OSAVI_mean_20251103_20251002,delta_OSAVI_mean_20251210_20251103,pct_OSAVI_mean_20251210_20251103,delta_SAVI2_mean_20251103_20251002,pct_SAVI2_mean_20251103_20251002,delta_SAVI2_mean_20251210_20251103,pct_SAVI2_mean_20251210_20251103
0,1,1,0.112061,47.717604,-0.255370,-73.614134,0.010226,1.166528,-0.146431,-16.511718,...,-2.734109,-38.091574,0.003649,0.550290,-0.121923,-18.283992,-0.000865,-18.147533,-0.000049,-1.252559
1,2,2,0.188731,120.754678,-0.248649,-72.067090,-0.047068,-5.233942,-0.048898,-5.737783,...,-1.477894,-27.794049,-0.014327,-2.105452,-0.054700,-8.211412,-0.000051,-1.168751,0.000626,14.432396
2,3,3,0.106408,50.351474,-0.182755,-57.517555,-0.046463,-5.212831,-0.039494,-4.674592,...,27497.579951,370487.353437,0.000125,0.017599,-0.710522,-99.931533,-0.000776,-14.899189,-0.004433,-99.986172
3,4,4,-0.046951,-29.595923,-0.015979,-14.306793,-0.195102,-22.074881,0.067883,9.856399,...,24945.104776,868036.329372,-0.126016,-18.003375,-0.573503,-99.923577,-0.000917,-17.113065,-0.004439,-99.987049
4,5,5,0.165937,76.740214,-0.260399,-68.137315,-0.065780,-7.533732,0.021049,2.607124,...,37023.334848,504878.287731,0.034816,5.057964,-0.722724,-99.941222,-0.000666,-12.572152,-0.004632,-99.988832
